<a href="https://colab.research.google.com/github/FelipeSotoG/Desafio-1-EDA/blob/main/WTABestFirst.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# WTA

En este tutorial explicaré como resolver un **magic square** usando grafos.

El magic square consiste en un cuadrito de 3x3 en el que hay que colocar números del 1 al 9 de tal manera que sus **filas**, **columnas** y **diagonales** sumen 15.

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/e/e4/Magicsquareexample.svg/800px-Magicsquareexample.svg.png" width="200">

### Modelamiento usando grafos

Para resolver el problema usando grafos, lo primero que debemos hacer es modelar su resolución. Es decir, debemos definir un **estado** de partida o **inicial**, el formato de los estados intermedios y un conjunto de reglas (**acciones**) que nos permitan ir pasando de un estado a otro.

Para nuestro problema, cada estado corresponderá a una **matriz de 3x3**.
Comenzaremos la resolución con una matriz vacía y las acciones corresponderán a colocar los números uno a uno. Es decir, una **acción** corresponderá a colocar un número X en una posición (i,j) de la matriz.

Para reducir la cantidad de acciones posibles a partir de un estado (sin dejar de explorar todas las alternativas), diremos que una acción consiste en colocar el siguiente número (comenzando de 1,2,3...), en una de las casillas disponibles. La varible `last_number` indica el valor del último número colocado en la matriz.

In [ ]:
from numpy.matrixlib.defmatrix import matrix
import numpy as np
import itertools
#formato de los estados
class State:
  id_iter = itertools.count()
  def __init__(self, matrix, weapons, prob, value):
    self.id = next(State.id_iter)
    self.matrix = matrix
    self.weapons = weapons
    self.prob = prob
    self.value = value
  def __lt__(self, other):
    return self.id < other.id
#matriz "vacía"
initial_state = State([[0,0,0],\
                       [0,0,0],\
                       [0,0,0]],
                       [5,2,1],
                      [[0.3,0.2,0.5],\
                       [0.1,0.6,0.5],\
                       [0.4,0.5,0.4]],
                       [5,10,20] )

class Action:
  def __init__(self,i,j):
    self.i=i; self.j=j

N=3 #número de filas/columnas

Luego definiré el método para pasar de un estado a otro usando una acción.
Observe que este método simplemente coloca en la posición (i,j) de la matriz, el siguiente número y actualiza la variable `last_number`.

In [ ]:
def transition(state, action):
  state.matrix[action.i][action.j] +=1 

Para saber **si un estado es final**, debemos revisar que el último número ingresado sea el 9 (N*N)

In [ ]:
def column_sum(lst):  
     return [sum(i) for i in zip(*lst)]

In [ ]:
def is_final_state(state):
  i=0
  for x in column_sum(state.matrix):
    if x == 0: return False
    i+=1
  i=0
  for x in state.matrix:
    if sum(x) != state.weapons[i]: return False
    i+=1
  return True

Para que un estado intermedio tenga posibilidades de convertirse en una solución al problema, podemos revisar que tanto sus filas, columnas como diagonales sumen 15 o menos. 

In [ ]:
def is_valid(state):
  i=0
  for x in state.matrix:
    if sum(x) > state.weapons[i]: return False
    i+=1
  return True

Definamos un método que nos permita obtener todas las **acciones válidas** a partir de un estado.

Para ello debemos revisar en la matriz cada casilla faltante y crear la acción correspondiente.

In [ ]:
def get_valid_actions(state):
  valid_actions = []
  for i in range(N):
    for j in range(N):
      valid_actions.append(Action(i,j))
  return valid_actions

## Resolución del problema

Una vez modelada la resolución del magic square, es hora de resolverlo. Para ello simplemente implementaremos un algoritmo de **búsqueda en profundidad** (dfs).

In [ ]:
import numpy as np

def heuristic_eval(state):
    minsum=0
    for j in range(N):
      arr=[]
      for i in range(N):
        if state.matrix[i][j] != 0: arr.append(1-pow(state.prob[i][j],state.matrix[i][j]))
      minsum+= state.value[j]*np.prod(arr) 
    return minsum

In [ ]:
import copy

from queue import PriorityQueue

def bfs(initial_state):
  q = PriorityQueue()
  q.put( (-1, copy.deepcopy(initial_state)) )
  iters = 0
  while not q.empty():
    iters += 1
    elem=q.get()
    state = elem[1]

    if is_valid(state) == False: continue #se descarta el estado

    if is_final_state(state): return state,iters #se encontró una solución!

    actions = get_valid_actions(state)

    for action in actions:
      new_state = copy.deepcopy(state)
      transition(new_state,action)
      q.put((heuristic_eval(new_state), new_state))


In [ ]:
state,iters=bfs(initial_state)
print(state.matrix)
print(heuristic_eval(state))
print("iteraciones:", iters)

[[0, 2, 3], [1, 0, 1], [0, 0, 1]]
1.4000000000000001
iteraciones: 14
